In [0]:
# Case 1 : Product is available and the customer care agent can help  with the quantity/ product information depending on the interaction with the customer
# Case 2 : Product is not available. Recommendation system will top 5 similar products  which are available in store along with product information and on hand quantity if required. So will help the customer agent to pitch the customer
# Allied products - Upselling - Applicable for both case 1 and 2
# Based on location information , we can recommend the closest store which will have the exact product

In [0]:
#Assumptions - to be included
# real time inventory data is not included -
# We can't provide assurance if the requested quantity by the customer will be fullfilled 

In [0]:
%pip install databricks-genai-inference
%pip install --upgrade typing_extensions
%pip install mlflow
%pip install tiktoken
%pip install dbdemos
%pip install --upgrade --force-reinstall databricks-vectorsearch

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 705.5/705.5 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.8/447.8 kB 49.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.3/409.3 kB 42.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.2/189.2 kB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 74.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.2/181.2 kB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.3/85.3 kB 9.1 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensio

In [0]:
%pip install --upgrade mlflow

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.


In [0]:
%pip install spacy

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 50.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.1/46.1 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.9/156.9 kB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 922.3/922.3 kB 65.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.0/493.0 kB 37.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.0/182.0 kB 25.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 98.5 MB/s eta 0:00:00
     ━━━━━━━━

In [0]:
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 65.2 MB/s eta 0:00:00

[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [0]:
dbutils.library.restartPython()

In [0]:
import mlflow
from mlflow.metrics.genai import EvaluationExample, make_genai_metric
import pandas as pd
from databricks_genai_inference import ChatCompletion
#import dbdemos
# dbdemos.install('llm-rag-chatbot')
from databricks.vector_search.client import VectorSearchClient

In [0]:
from pyspark.sql.window import *
from pyspark.sql.functions import *
import re
import spacy

In [0]:
data_inventory_filter = spark.read.table("mad_angle.default.harmonized_retailer_inventory_store_delta_filter_with_id2")
print(data_inventory_filter.columns)

catalog_name = "mad_angle"
schema_name = "default"
source_table_name = "harmonized_retailer_inventory_store_delta_filter_with_id2_top100_with_cat"
source_table_fullname = f"{catalog_name}.{schema_name}.{source_table_name}"

window = Window.partitionBy().orderBy(data_inventory_filter['on_hand_quantity'].desc())

data_inventory_top_100  = spark.read.table(source_table_fullname)





['retailer', 'product_id', 'store_id', 'supplier', 'product', 'product_key', 'upc', 'chain', 'store', 'store_number', 'store_city', 'store_state', 'store_zip', 'store_latitude', 'store_longitude', 'date_key', 'on_hand_quantity', 'id']


In [0]:
generation_config = {
  "temperature": 1,
  "top_p": 0.95,
  "top_k": 0,
  "max_output_tokens": 8192,
}
safety_settings = [
  {
    "category": "HARM_CATEGORY_HARASSMENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_HATE_SPEECH",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
]


In [0]:

def get_input_list(dummy_arg):
  dummy_arg_list = []
  if type(dummy_arg) == list:
    dummy_arg_list.append(dummy_arg)
  elif type(dummy_arg) == dict:
    if dummy_arg.get('inputs'):
      dummy_arg_list.append(dummy_arg['inputs'])
    else:
      dummy_arg_list.append([dummy_arg])
  else:
    for i in dummy_arg.index:
      if type(dummy_arg['inputs'][i]) == list:
        tmparg = dummy_arg['inputs'][i]
      else:
        tmparg = [dummy_arg['inputs'][i]]
      dummy_arg_list.append(tmparg)
  return dummy_arg_list


def dbrxmodel(dummy_arg):
  dummy_arg_op_list = []
  dummy_arg_in_list = get_input_list(dummy_arg)
  for tmparg_ in dummy_arg_in_list:
    op = ChatCompletion.create(model="databricks-dbrx-instruct",messages=tmparg_,max_tokens=4096).message
    dummy_arg_op_list.append({"output":op})
  return pd.DataFrame(dummy_arg_op_list)



question_list = ["What can be your product recommendation(product,product key) and product information details in the same category to product 3539  in store name ending with 15536 based on inventory available","What is the on hand inventory of product 0995 on 9th July 2022 in store 00128"]
question_list_new = []
for quest in question_list:
    question_list_new.append({"role": 'user',"content":quest})
eval_df_dbrx = pd.DataFrame({"inputs":question_list_new})

display(eval_df_dbrx)


inputs
"List(What can be your product recommendation(product,product key) and product information details in the same category to product 3539 in store name ending with 15536 based on inventory available, user)"
"List(What is the on hand inventory of product 0995 on 9th July 2022 in store 00128, user)"


Additional Data Preparation

In [0]:
# from pyspark.sql.functions import col, create_map, lit
# from itertools import chain

# col_list = ['product_key']
# for col in col_list:
#     unique_list_col = [i[col] for i in data_inventory_top_100.select(col).distinct().collect()]
#     print(unique_list_col)
#     print(len(unique_list_col))
#     dict_label_col = {}
#     dict_label_col_sub = {}
    
#     for  i in range(len(unique_list_col)):
#         dict_label_col[str(unique_list_col[i])]= "Category_"+str(i%2)
       

#     print(dict_label_col)
#     print(dict_label_col_sub)
#     globals()["udf_label"+col] = udf(lambda col: dict_label_col.get(col), StringType())
    
#     data_inventory_top_100 = data_inventory_top_100.withColumn("Category_"+col,globals()["udf_label"+col](col))
   
 
    

        

In [0]:
# data_inventory_top_100 = data_inventory_top_100.drop('rank')

In [0]:
# source_table_name2 = "harmonized_retailer_inventory_store_delta_filter_with_id2_top100_with_cat"
# source_table_fullname2 = f"{catalog_name}.{schema_name}.{source_table_name2}"
# data_inventory_top_100.write.format("delta").option("delta.enableChangeDataFeed", "true").saveAsTable(source_table_fullname2)

In [0]:
system_prompt = 'Instructions: Given following SQL schema, records sample & data dictionary, your job is to write ONLY SQL query with proper syntax given a user’s request. Do not join these tables unless absolutely necessary!. Please pick the table for generating query only if the column is present in that table schema. Generate subquery if required to answer complex questions.Also if the user mentions The phrase "across the years" it means "for every year".The phrase "across categories" means "for every categories".The phrase "across retailer level" means "for every retailers".The phrase "across brand level" means "for every brands".The phrase "sales uplift" means "percentage increase in sales or revenue due to specific promotional activities".The term "overall spends" means "total spends".The term "drainers and drivers" means "changes experienced by different brands and pack types over time".The term "current period" refers to the "latest year in the data" and "previous period" refers to the "previous years other than latest year in the data".Whenever you are doing a group by in SQL query make sure to consider that column in the select part of the query has context menu.Whenever you are doing a group by in SQL query make sure to consider that column in the select part of the query.\
Context: Table: mad_angle.default.harmonized_retailer_inventory_store_delta_filter_with_id2_top100_with_cat \
Columns : `retailer`,`product_id`,`store_id`,`supplier`,`product`,`product_key`,`upc`,`chain`, \
          `store`,`store_number`,`store_city`,`store_state`,`store_zip`,`store_latitude`,\
          `store_longitude`,`date_key`,`on_hand_quantity`,`id`,`Category_product_key`\
Dictionary for mad_angle.default.harmonized_retailer_inventory_store_delta_filter_with_id2_top100_with_cat : In this context the column "retailer" refers to the retailer for example retailer 1, retailer 2,  etc, the column "product_id" refers to the product identifier information,the column "store_id" refers to the store identifier information, the column "supplier" refers to the supplier name,the column "product" denotes the name of the product, the column "product_key" refers to the product key information,the column "upc" represents the universal product code of the product ,  the column "chain" represents the retail supermarket chain or banner information,the column "store" represents the store information, the column "store_number" represents the store identifier,the column "store_city" is  name of the city in which store exists,the column "store_state" is  name of the state in which store exists ,the column "store_zip" is   zipcode of the city in which store exists ,the column "store_latitude" is  latitude information of the store ,the column "store_longitude" is  longitude information of the store ,the column "date_key" is  date in which the information was collected, the column "on_hand_quantity" is  the on hand quantity of the inventory available in the store for that particular product , the column "id" is  the identifier denoting the unique  combination of product store information in a store, the column "Category_product_key" is  the identifier denoting the category of product information in a store\
Data for mad_angle.default.harmonized_retailer_inventory_store_delta_filter_with_id2_top100_with_cat: \
1 | `RETAILER 6`,8699263674839736531,521626801377406987,`SUPPLIER 06`,`PRODUCT 4836`,`4836`,`0000000004836`,`RETAILER 6`,`STORE 30469`,`04731`,`CITY 09257`,`SC`,`29203`,34.1,-81.04,`2022-07-13`,11211,`86992636748397365315216268013774069872022-07-13`,`Category_0`\
2 | `RETAILER 6`,8699263674839736531,521626801377406987,`SUPPLIER 06`,`PRODUCT 4836`,`4836`,`0000000004836`,`RETAILER 6`,`STORE 30469`,`04731`,`CITY 09257`,`SC`,`29203`,34.1,-81.04,`2022-06-07`,11950,	`86992636748397365315216268013774069872022-06-07`,`Category_0`\
3 | `RETAILER 4`,5373936945841831885,45926792719034451,`SUPPLIER 06`,`PRODUCT 5516`,`5516`,`000000000551`,`RETAILER 4`,`STORE 0564`,`0029`,`CITY 02294`,`NJ`,`0801`,39.82,-75.35,`2022-08-28`,188,`5373936945841831885459267927190344512022-08-28`,`Category_1`\
4 | `RETAILER 4`,3493016077731501006,2544351317611134725,`SUPPLIER 06`,`PRODUCT 5051`,`5051`,`0000000005051`,`RETAILER 4`,`STORE 11522`,`00659`,`CITY 32007`,`OR`,`97321`,44.65,-123.14,`2022-08-28`,48,`349301607773150100625443513176111347252022-08-28`,`Category_1`\
5 | `RETAILER 4`,860960474656778313,2544351317611134725,`SUPPLIER 06`,`PRODUCT 4994`,`4994`,`0000000004994`,`RETAILER 4`,`STORE 11522`,`00659`,`CITY 32007`,`OR`,`97321`,44.65,-123.14,`2022-08-28`,23,	`86096047465677831325443513176111347252022-08-28`,`Category_1`'




Recommendation 

In [0]:
mlflow.set_experiment("/Workspace/Shared/Mad Angle /mlflow_experiments/dbrx_solution_prototype_v33")
i=0
pred_dbx_final = pd.DataFrame([])
while i< eval_df_dbrx.shape[0]:
    with mlflow.start_run(run_name='log_dbrx_model'):
        dbrx_model_info = mlflow.pyfunc.log_model(artifact_path='dbrx_model',python_model=dbrxmodel,
                                                #signature=signature_dbrx,
                                                input_example=pd.DataFrame({'inputs':[{"role": "system","content": f"{system_prompt}"},
                                                                eval_df_dbrx['inputs'].iloc[i]]})
                                                )
        results = mlflow.evaluate(model=dbrx_model_info.model_uri,data=eval_df_dbrx,
                            model_type="question-answering",
                            evaluators="default",
                            #   extra_metrics=[
                            #       answer_accessibility
                            #       ],
                                )
    dbrx_model = mlflow.pyfunc.load_model(dbrx_model_info.model_uri)

    pred_dbrx = dbrx_model.predict({'inputs':[{"role": "system","content": f"{system_prompt}"},
                                                                eval_df_dbrx['inputs'].iloc[i]]})
    print('##################### output from DBRX #####################\n')
    display(pred_dbrx)
    pred_dbx_final = pred_dbx_final.append(pred_dbrx)
    i+=1

2024/05/06 23:16:34 INFO mlflow.tracking.fluent: Experiment with name '/Workspace/Shared/Mad Angle /mlflow_experiments/dbrx_solution_prototype_v33' does not exist. Creating a new experiment.
2024/05/06 23:16:41 WARNING mlflow.utils.requirements_utils: Failed to run predict on input_example, dependencies introduced in predict are not captured.
PySparkRuntimeError()Traceback (most recent call last):


  File "/local_disk0/.ephemeral_nfs/envs/pythonEnv-baecf587-f692-4ed4-bd68-2da6de781a56/lib/python3.10/site-packages/mlflow/utils/_capture_modules.py", line 135, in _load_pyfunc_patch
    model.predict(input_example, params=params)


  File "/local_disk0/.ephemeral_nfs/envs/pythonEnv-baecf587-f692-4ed4-bd68-2da6de781a56/lib/python3.10/site-packages/mlflow/pyfunc/model.py", line 543, in predict
    return self.python_model.predict(


  File "/local_disk0/.ephemeral_nfs/envs/pythonEnv-baecf587-f692-4ed4-bd68-2da6de781a56/lib/python3.10/site-packages/mlflow/pyfunc/model.py", line 162, in predi

Uploading artifacts:   0%|          | 0/10 [00:00<?, ?it/s]

2024/05/06 23:16:41 INFO mlflow.store.artifact.cloud_artifact_repo: The progress bar can be disabled by setting the environment variable MLFLOW_ENABLE_ARTIFACTS_PROGRESS_BAR to false


2024/05/06 23:16:43 INFO mlflow.store.artifact.artifact_repo: The progress bar can be disabled by setting the environment variable MLFLOW_ENABLE_ARTIFACTS_PROGRESS_BAR to false
2024/05/06 23:16:44 INFO mlflow.models.evaluation.base: Evaluating the model with the default evaluator.
2024/05/06 23:16:44 INFO mlflow.models.evaluation.default_evaluator: Computing model predictions.
2024/05/06 23:16:48 INFO mlflow.models.evaluation.default_evaluator: Testing metrics on first row...
2024/05/06 23:16:48 WARNING mlflow.metrics.metric_definitions: Failed to load 'toxicity' metric (error: ModuleNotFoundError("No module named 'evaluate'")), skipping metric logging.
2024/05/06 23:16:48 WARNING mlflow.models.evaluation.default_evaluator: Did not log builtin metric 'toxicity' because it returned None.
2024/05/06 23:16:48 WARNING mlflow.metrics.metric_definitions: Failed to import textstat for flesch kincaid metric, skipping metric logging. Please install textstat using 'pip install textstat'.
2024/05

2024/05/06 23:16:50 INFO mlflow.store.artifact.artifact_repo: The progress bar can be disabled by setting the environment variable MLFLOW_ENABLE_ARTIFACTS_PROGRESS_BAR to false


...

2024/05/06 23:17:00 INFO mlflow.store.artifact.artifact_repo: The progress bar can be disabled by setting the environment variable MLFLOW_ENABLE_ARTIFACTS_PROGRESS_BAR to false
2024/05/06 23:17:01 INFO mlflow.models.evaluation.base: Evaluating the model with the default evaluator.
2024/05/06 23:17:01 INFO mlflow.models.evaluation.default_evaluator: Computing model predictions.
2024/05/06 23:17:04 INFO mlflow.models.evaluation.default_evaluator: Testing metrics on first row...
2024/05/06 23:17:04 WARNING mlflow.metrics.metric_definitions: Failed to load 'toxicity' metric (error: ModuleNotFoundError("No module named 'evaluate'")), skipping metric logging.
2024/05/06 23:17:04 WARNING mlflow.models.evaluation.default_evaluator: Did not log builtin metric 'toxicity' because it returned None.
2024/05/06 23:17:04 WARNING mlflow.metrics.metric_definitions: Failed to import textstat for flesch kincaid metric, skipping metric logging. Please install textstat using 'pip install textstat'.
2024/05

2024/05/06 23:17:06 INFO mlflow.store.artifact.artifact_repo: The progress bar can be disabled by setting the environment variable MLFLOW_ENABLE_ARTIFACTS_PROGRESS_BAR to false


##################### output from DBRX #####################



output
"Here is the SQL query to get the on-hand inventory of product 0995 on 9th July 2022 in store 00128: ``` SELECT on_hand_quantity FROM mad_angle.default.harmonized_retailer_inventory_store_delta_filter_with_id2_top100_with_cat WHERE product_key = '0995' AND store_number = '00128' AND date_key = '2022-07-09'; ``` This query selects the `on_hand_quantity` column from the `mad_angle.default.harmonized_retailer_inventory_store_delta_filter_with_id2_top100_with_cat` table, and filters the results to only include rows where the `product_key` is '0995', the `store_number` is '00128', and the `date_key` is '2022-07-09'. This should give you the on-hand inventory of product 0995 on 9th July 2022 in store 00128. Note: The `product_key` column in the table schema is used to refer to the product identifier information, and the `store_number` column is used to refer to the store identifier information. The `date_key` column is used to refer to the date in which the information was collected. The `on_hand_quantity` column is used to refer to the on-hand quantity of the inventory available in the store for that particular product."


/home/spark-baecf587-f692-4ed4-bd68-2d/.ipykernel/37427/command-191870854589094-786705813:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pred_dbx_final = pred_dbx_final.append(pred_dbrx)


In [0]:
pred_dbx_final.display()

output
"Here is the SQL query to get the product recommendation and product information details in the same category as product 3539 in the store with a name ending with 15536, based on the inventory available: ```vbnet SELECT product, product_key, supplier, on_hand_quantity FROM mad_angle.default.harmonized_retailer_inventory_store_delta_filter_with_id2_top100_with_cat WHERE Category_product_key = ( SELECT Category_product_key FROM mad_angle.default.harmonized_retailer_inventory_store_delta_filter_with_id2_top100_with_cat WHERE product_key = '3539' AND store LIKE '%15536' ) AND store LIKE '%15536' AND product_key!= '3539' ORDER BY on_hand_quantity DESC LIMIT 5; ``` This query will return the top 5 products in the same category as product 3539 with the highest on-hand quantity in the store with a name ending with 15536, excluding product 3539 itself. The result will include the product name, product key, supplier, and on-hand quantity."
"Here is the SQL query to get the on-hand inventory of product 0995 on 9th July 2022 in store 00128: ``` SELECT on_hand_quantity FROM mad_angle.default.harmonized_retailer_inventory_store_delta_filter_with_id2_top100_with_cat WHERE product_key = '0995' AND store_number = '00128' AND date_key = '2022-07-09'; ``` This query selects the `on_hand_quantity` column from the `mad_angle.default.harmonized_retailer_inventory_store_delta_filter_with_id2_top100_with_cat` table, and filters the results to only include rows where the `product_key` is '0995', the `store_number` is '00128', and the `date_key` is '2022-07-09'. This should give you the on-hand inventory of product 0995 on 9th July 2022 in store 00128. Note: The `product_key` column in the table schema is used to refer to the product identifier information, and the `store_number` column is used to refer to the store identifier information. The `date_key` column is used to refer to the date in which the information was collected. The `on_hand_quantity` column is used to refer to the on-hand quantity of the inventory available in the store for that particular product."


In [0]:
i=0
while i<pred_dbx_final.shape[0]:
    nlp = spacy.load("en_core_web_sm")
    doc = nlp(pred_dbx_final['output'].iloc[i])
    sql_pattern =  r"```(.*?)```|(SELECT(.*?);)"
    matches = re.search(sql_pattern, doc.text,re.DOTALL)
    try:
            sql_query = matches.group(1).strip()
            sql_query = sql_query.replace("\n"," ")
            sql_query = sql_query.replace(";","")
            sql_query = sql_query.removeprefix("vbnet")
            sql_query = sql_query.removeprefix("sql")
            print("Extracted SQL Query:")
            print(sql_query)
    except:
                print("No SQL query found in the text.")

    # Using PySpark to execute the SQL query
    df = spark.sql(sql_query)

    # Show the results
    df.show()
    i+=1

Extracted SQL Query:
 SELECT product, product_key, supplier, on_hand_quantity FROM mad_angle.default.harmonized_retailer_inventory_store_delta_filter_with_id2_top100_with_cat WHERE Category_product_key = (   SELECT Category_product_key   FROM mad_angle.default.harmonized_retailer_inventory_store_delta_filter_with_id2_top100_with_cat   WHERE product_key = '3539' AND store LIKE '%15536' ) AND store LIKE '%15536' AND product_key!= '3539' ORDER BY on_hand_quantity DESC LIMIT 5
+------------+-----------+-----------+----------------+
|     product|product_key|   supplier|on_hand_quantity|
+------------+-----------+-----------+----------------+
|PRODUCT 0160|       0160|SUPPLIER 06|         15150.0|
|PRODUCT 0160|       0160|SUPPLIER 06|         13485.0|
+------------+-----------+-----------+----------------+

Extracted SQL Query:
SELECT on_hand_quantity FROM mad_angle.default.harmonized_retailer_inventory_store_delta_filter_with_id2_top100_with_cat WHERE product_key = '0995' AND store_number

In [0]:
import mlflow 
from pyspark.sql.functions import struct, col 
logged_model = 'runs:/247e266185b5478283c0468f7bdd77e4/dbrx_model' 
# Load model as a Spark UDF. 
# loaded_model = mlflow.pyfunc.spark_udf(spark, model_uri=logged_model) 
#  Predict on a Spark DataFrame. df.withColumn('predictions', loaded_model(struct(*map(col, df.columns))))

In [0]:
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.ml.functions import *

In [0]:
## show me the on hand quantity of product starting/ending with 3563 on 2022-11-30 and STORE 00076

In [0]:
# Attributes of every product - it comes under category, price point , disposition, size/quantity ...etc
# Allied products - recommend  500 ml  instead of 2 250 ml either we dont have 250 ml sufficient quantity / see if we can conceive this opportunity to increase the share of wallet
# complimentary products - recommend bread and butter together - we need LLM to suggest these

## customer is very loyal to Manufacturer , so in recommendation , the products from Manufacturer either allied - optional
